# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here
orders=pd.read_csv("Orders.csv")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [9]:
# your code here
#amount_spent for unique customers
clients=orders.groupby(["CustomerID", "Country"]).agg({"amount_spent": "sum"}).reset_index()
clients.head()

,CustomerID,Country,amount_spent
0,12346,United Kingdom,77183.60
1,12347,Iceland,4310.00
2,12348,Finland,1797.24
3,12349,Italy,1757.55
4,12350,Norway,334.40


In [21]:
#Qualtile calculation
quantiles=np.quantile(clients["amount_spent"], q=[0.75,0.95])
quantiles

array([1661.195, 5774.278])

In [22]:
clients.tail()

,CustomerID,Country,amount_spent
4342,18280,United Kingdom,180.60
4343,18281,United Kingdom,80.82
4344,18282,United Kingdom,178.05
4345,18283,United Kingdom,2094.88
4346,18287,United Kingdom,1837.28


In [24]:
def customer_type (row):
        if row["amount_spent"]>=quantiles[1]:
            return "VIP"
        elif row["amount_spent"]>=quantiles[0] and row["amount_spent"]<quantiles[1]:
            return "Preferred"
        else:
            return "Smaller"

In [26]:
clients["Type"]=clients.apply(customer_type, axis=1)
clients.head()

,CustomerID,Country,amount_spent,Type
0,12346,United Kingdom,77183.60,VIP
1,12347,Iceland,4310.00,Preferred
2,12348,Finland,1797.24,Preferred
3,12349,Italy,1757.55,Preferred
4,12350,Norway,334.40,Smaller


In [48]:
seg=clients.groupby(["Country", "Type"]).agg({"CustomerID": "count"}).reset_index()
seg

,Country,Type,CustomerID
0,Australia,Preferred,3
1,Australia,Smaller,5
2,Australia,VIP,1
3,Austria,Preferred,1
4,Austria,Smaller,10
...,...,...,...
70,United Arab Emirates,Smaller,2
71,United Kingdom,Preferred,756
72,United Kingdom,Smaller,2987
73,United Kingdom,VIP,178


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [46]:
seg[seg["Type"]=="VIP"].max()

Country       United Kingdom
Type                     VIP
CustomerID               178
dtype: object

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [72]:
# your code here
seg2=seg[(seg["Type"] =="VIP") | (seg["Type"]=="Preferred")]
seg2.groupby(["Country"]).agg({"CustomerID": "sum"}).reset_index().max()

Country       United Kingdom
CustomerID               934
dtype: object